In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc("font", family="Malgun Gothic")

In [ ]:
# 180일 이상 구매데이터가 없는 경우 신규고객또는 비활동고객이라고 정의.
# 구매상품 데이터
import pandas as pd
from datetime import datetime
dt_parser = lambda x: datetime.strptime(x, "%Y%m%d") 
df4 = pd.read_csv('/content/drive/MyDrive/1차팀플/dataset/구매상품TR.txt', encoding='CP949',
                 parse_dates=['구매일자'], date_parser=dt_parser)
df4.shape

In [ ]:
def drop_dup(df):
    df.drop_duplicates(subset='영수증번호', keep='first', inplace=False, ignore_index=False)
    return df  

In [ ]:
df_last_day = df4[['고객번호', '구매일자']]
a = df_last_day.groupby(['고객번호','구매일자'])[['count_s']].count()
a = a.reset_index()
a1 = a.drop_duplicates(subset='고객번호', keep='last', inplace=False, ignore_index=False)
a1['last_day'] = '2015-12-31'
a1['last_day'] = a1['last_day'].astype('datetime64[ns]')
a1['days'] = a1['last_day'] - a1['구매일자']
a1['days']= (a1['days']/ np.timedelta64(1,'D')).astype(int)
data_a1 = a1[a1['days'] > 180]
data_a1 = data_a1[['고객번호','days']]
data_a1.columns = ['고객번호', '구매간격(days)']

In [ ]:
# 구매간격 구하는 사용자함수
def diff_days(df):
    df2 = pd.pivot_table(df,index=['고객번호','구매일자'])
    df2 = df2.reset_index()
    df2 = df2[['고객번호','구매일자']]
  
    dd = pd.DataFrame()
    for i in range(1,19384):
        try:
            cond = df2['고객번호'] == i
            d = df2[cond].diff()
            dd = dd.append(d)
        except: print(i)
    dd = dd['구매일자'].dropna()
    
    dd['구매일자']= (dd['구매일자']/ np.timedelta64(1,'D')).astype(int)
    df2['구매간격(days)'] = dd
    return df2

In [ ]:
dd = pd.DataFrame()
for i in range(1,19384):
    try:
        cond = df2['고객번호'] == i
        d = df2[cond].diff()
        dd = dd.append(d)

    except: print(i)

In [ ]:
aa = pd.Series()

for i in range(1,19384):
    try:
        cond = df2['고객번호'] == i
        d = df2[cond].diff()
        d = d.dropna()
        d = d.max()
        aa = aa.append(d)
    except:print(i)  

In [ ]:
def diff_max(df):
    aa = pd.Series()
    for i in range(1,19384):
        try:
            cond = df['고객번호'] == i
            d = df[cond].diff()
            d = d.dropna()
            d = d.max()
            aa = aa.append(d)
        except:print(i)
    aa = pd.DataFrame(aa)
    aa = aa.reset_index(drop=True)
    aa.columns = ['구매일자']

    cond = aa != 0
    aa = aa[cond]
    aa = aa.dropna()

    aa['구매일자']= (aa['구매일자']/ np.timedelta64(1,'D')).astype(int)
    aa = aa.reset_index(drop=True)
    aa['고객번호'] = aa.index + 1
    aa = aa.rename({'구매일자':'구매간격(days)'}, axis=1)
    return aa

In [ ]:
a = diff_max(df2)
a

In [ ]:
aa['구매일자']= (aa['구매일자']/ np.timedelta64(1,'D')).astype(int)
aa = aa.reset_index(drop=True)
aa['고객번호'] = aa.index + 1
aa = aa.rename({'구매일자':'구매간격(days)'}, axis=1)
aa.sort_values(by ='구매간격(days)')

In [ ]:
def outliers_iqr(data):
    q1, q3 = np.percentile(data, [25,75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)

    return lower_bound, upper_bound

lower_wis, upper_wis = outliers_iqr(dd['구매일자']) #구매 기간의 간격 값
# (-2.0, 6.0)
lower_wis, upper_wis = outliers_iqr(aa['구매간격(days)'])
# (-9.0, 39.0)

In [ ]:
aa['구매간격(days)'].plot(kind='box')

In [ ]:
dd = dd['구매일자'].dropna()
import numpy as np
dd['구매일자']= (dd['구매일자']/ np.timedelta64(1,'D')).astype(int)
df2['구매간격(days)'] = dd

In [ ]:
df4['year'] = df4['구매일자'].dt.year
df4['month'] = df4['구매일자'].dt.month
df4['day'] = df4['구매일자'].dt.day

In [ ]:
def drop_dup(df):
    df.drop_duplicates(subset='영수증번호', keep='first', inplace=False, ignore_index=False)
    df = df.reset_index(drop=True)
    return df

def diff_days(pi): 
    dd = pd.DataFrame()
    for i in range(1,19384):
        try:
            cond = pi['고객번호'] == i
            d = pi[cond].diff()
            dd = dd.append(d)

        except: print(i)

    dd = dd['구매일자'].dropna()
    dd = pd.DataFrame(dd)
    dd['구매일자']= (dd['구매일자']/ np.timedelta64(1,'D')).astype(int)
    pi['구매간격(days)'] = dd

    return pi

In [ ]:
# df2014_1_drop = drop_dup(df2014_1)
pi = pd.pivot_table(df2014_1_drop,index=['고객번호','구매일자'],columns=['구매금액'], aggfunc='sum')
pi = pi.reset_index()
pi1 = pi[['고객번호','구매일자']]

dd = pd.DataFrame()
for i in range(1,19384):
    try:
        cond = pi1['고객번호'] == i
        d = pi1[cond].diff()
        dd = dd.append(d)

    except: print(i)

dd = dd['구매일자'].dropna()

dd['구매일자']= (dd['구매일자']/ np.timedelta64(1,'D')).astype(int)
pi['구매간격(days)'] = dd
diff_days(pi)

In [ ]:
pi[pi['구매간격(days)'] < 180]

In [ ]:
#----------------------------------------------------------------------------------

In [ ]:
# !pip3 install pickle5 
import pickle5 as pickle
with open('/content/drive/MyDrive/1차팀플/dataset/draft.pkl', "rb") as fh:
    data= pickle.load(fh)

In [ ]:
df = data[['영수증번호','중분류명', '소분류명', '년', '분기', '성별','구매금액', '제휴사', '고객번호']]
df.to_pickle('/content/drive/MyDrive/1차팀플/dataset/상품별분석.pkl')

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/1차팀플/dataset/상품별분석.pkl')

In [ ]:
# 중분류명 분기별 신규/기존 고객별로 가장많이 판매된 상품 비교
new_cust = pd.read_pickle('/content/drive/MyDrive/1차팀플/dataset/new고객번호.pkl')

In [ ]:
newlist = new_cust['고객번호'].values

In [ ]:
# 일단 년도별로 나누기
df2014 = df[df['년']==2014]
df2015 = df[df['년']==2015]
df2014_1= df2014[df2014['분기']==1]
df2014_2= df2014[df2014['분기']==2]
df2014_3= df2014[df2014['분기']==3]
df2014_4= df2014[df2014['분기']==4]
df2015_1= df2015[df2015['분기']==1]
df2015_2= df2015[df2015['분기']==2]
df2015_3= df2015[df2015['분기']==3]
df2015_4= df2015[df2015['분기']==4]

In [ ]:
def comparison_new(df, newlist):
    df_new = pd.DataFrame()
    df_exi = pd.DataFrame()
    for i in range(1,19384):
        if i in newlist:
            cond2 = df['고객번호'] == i
            new = df[cond2]
            df_new = df_new.append(new)
        else:
            cond1 = df['고객번호'] == i
            exi = df[cond1]
            df_exi = df_exi.append(exi)
    return df_new, df_exi

In [ ]:
# 2014_1분기~2015 기존, 신규고객 데이터
df2014_1_new, df2014_1_exi = comparison_new(df2014_1, newlist)
df2014_2_new, df2014_2_exi = comparison_new(df2014_2, newlist)
df2014_3_new, df2014_3_exi = comparison_new(df2014_3, newlist)
df2014_4_new, df2014_4_exi = comparison_new(df2014_4, newlist)
df2015_1_new, df2015_1_exi = comparison_new(df2015_1, newlist)
df2015_2_new, df2015_2_exi = comparison_new(df2015_2, newlist)
df2015_3_new, df2015_3_exi = comparison_new(df2015_3, newlist)
df2015_4_new, df2015_4_exi = comparison_new(df2015_4, newlist)

In [ ]:
# 빈 DataFrame 만드는 사용자함수
def empty_df(index_a, columns_a):
  empty = pd.DataFrame(index=index_a,
                      columns = columns_a)
  return empty

In [ ]:
list_index = ['1순위','2순위','3순위','4순위','5순위','6순위','7순위','8순위','9순위','10순위']
list_columns = [['2014_1','2014_1','2014_1_매출비율','2014_1_매출비율','2014_2','2014_2','2014_2_매출비율','2014_2_매출비율',
                 '2014_3','2014_3','2014_3_매출비율','2014_3_매출비율','2014_4','2014_4','2014_4_매출비율','2014_4_매출비율',
                 '2015_1','2015_1','2015_1_매출비율','2015_1_매출비율','2015_2','2015_2','2015_2_매출비율','2015_2_매출비율',
                 '2015_3','2015_3','2015_3_매출비율','2015_3_매출비율','2015_4','2015_4','2015_4_매출비율','2015_4_매출비율'],
                ['기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객',
                 '기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객','기존고객','신규고객']]

rank = empty_df(list_index, list_columns)
rank = rank.T

In [ ]:
def pay_rank_10(df):
    g = df.groupby(['중분류명','성별']).agg({'소분류명':'count','구매금액':'sum'})
    g = g.reset_index()
    g = g.sort_values(by='구매금액', ascending=False)[:10]
    return g

def pay_raito_10_values(df):
    g = pay_rank_10(df)
    v = (g['구매금액'].values/m_sum)*100
    return v

def input_raito10(df_new, df_exi, rank_df,index1='2014_1_매출비율',index2='기존고객'):
    item_10_new = pay_raito_10_values(df_new)
    item_10_exi = pay_raito_10_values(df_exi)
    rank_df.loc[index1].loc['기존고객'] = item_10_exi
    rank_df.loc[index1].loc['신규고객'] = item_10_new
    return rank_df

def pay_rank_10_values(df):
    g = pay_rank_10(df)
    return g['중분류명'].values

def input_values10(df, rank_df,index1='2014_1',index2='기존고객'):
    item_10 = pay_rank_10_values(df)
    rank_df.loc[index1].loc[index2] = item_10
    return rank_df

def input_rank(rank, df_new, df2_exi, index_name):
    rank = input_values10(df_new, rank,index1=index_name,index2='신규고객')
    rank = input_values10(df2_exi, rank,index1=index_name,index2='기존고객')
    return rank

In [ ]:
# 전체 금액
m_sum = df2014_1_new['구매금액'].sum() + df2014_1_exi['구매금액'].sum() 

In [ ]:
# 구매비율 구하는 식
# 1분기 전체 구매금액 = m_sum
# 해당 상품 구매 금액 합/1분기 전체 구매 금액
(df2014_1_new_g['구매금액'].values/m_sum)*100

In [ ]:
rank = input_raito10(df2014_1_new, df2014_1_exi, rank,index1='2014_1_매출비율')
rank = input_raito10(df2014_2_new, df2014_2_exi, rank,index1='2014_2_매출비율')
rank = input_raito10(df2014_3_new, df2014_3_exi, rank,index1='2014_3_매출비율')
rank = input_raito10(df2014_4_new, df2014_4_exi, rank,index1='2014_4_매출비율')
rank = input_raito10(df2015_1_new, df2015_1_exi, rank,index1='2015_1_매출비율')
rank = input_raito10(df2015_2_new, df2015_2_exi, rank,index1='2015_2_매출비율')
rank = input_raito10(df2015_3_new, df2015_3_exi, rank,index1='2015_3_매출비율')
rank = input_raito10(df2015_4_new, df2015_4_exi, rank,index1='2015_4_매출비율')

In [ ]:
rank1 = rank.reset_index(level=1)

In [ ]:
rank_ratio = rank1.loc[['2014_1_매출비율','2014_2_매출비율', '2014_3_매출비율', '2014_4_매출비율', '2015_1_매출비율', 
                        '2015_2_매출비율', '2015_3_매출비율', '2015_4_매출비율']]

In [ ]:
def show_g(df, y='합계'):
    df = df[[y]]
    df = df.reset_index()

    import matplotlib.pyplot as plt
    import seaborn as sns

    plt.figure(figsize=(20,6))
    plt.rc('font', family='NanumGothic')
    sns.barplot(data = df, x='구분',hue='고객', y=y)

    plt.show()

In [ ]:
rank_ratio_sum = rank_ratio.drop('level_1', axis=1)
rank_ratio['합계'] = rank_ratio_sum.sum(axis=1).values
rank_ratio1 = rank_ratio.reset_index()
rank_ratio1 = rank_ratio1.rename({'index':'구분','level_1':'고객'},axis=1)
rank_ratio1 = rank_ratio1.set_index(['구분','고객'])
show_g(rank_ratio1, y='합계')

In [ ]:
show_g(rank_ratio1, y='8순위')

In [ ]:
g = rank_item[['level_1','10순위']]
g.sort_values(by='level_1')

In [ ]:
rank_item = rank1.loc[['2014_1','2014_2', '2014_3', '2014_4', '2015_1', '2015_2', '2015_3', '2015_4']]
rank_item.sort_values(by='level_1')

In [ ]:
# 상품분류별 기존고객/신규고객 매출비율

In [ ]:
def map_item(x):
    if x in ['가구', '가구/인테리어샵', '벽지류', '브랜드침구', '상', '생활잡화', '성인침구',
       '성인침구류', '소가구', '수납용품', '수도용품', '수예시즌', '시계', '아동침구', '아동침구류',
       '원예용품', '의류잡화', '의자', '의자/상', '장식용품', '침구/수예', '커튼류', '테이블']:
         x = '가구/인테리어'
    elif x in ['DVD', 'MP3/PMP', 'NAVI/하이패스', 'PC', 'TV', 'VIDEOGAME', '가습기',
       '건강가전', '건강디지털기기', '공기청정기', '기기', '기타음향기기', '기타주방가전', '김치냉장고',
       '냉장고', '대형가전', '믹서기', '밥솥', '생활소형가전', '선풍기', '세탁기', '소형가전',
       '소형음향기기', '시즌/건강가전', '안전용품', '에어컨', '오디오', '이동통신', '장착용품', '전구',
       '전기면도기', '전기요/장판', '전기용품', '전기전자', '전기포트', '전자/전기', '전자악기',
       '전자학습기', '전화기/팩스', '정수기', '조리기', '조명/전구', '조명기구', '주방가전', '주방소형가전',
       '청소기', '카메라/캠코더', '커피메이커/머신', '컴퓨터주변기기', '통신/컴퓨터', '홈시어터', '히터기']:
         x = '가전'
    elif x in ['EDUCATION 1', 'EDUCATION 2', '교육', '교육보조재', '남아', '노트', '노트/지류',
       '문구/팬시', '문화용품', '미술용품', '서적', '아동', '악기', '영상', '예능학용품', '오피스용품',
       '유/아동서적', '유아용품', '음반', '음반영상', '일반서적', '장식용품', '파티/팬시',
       '파티/팬시/시즌', '파티/팬시용품', '필기도구']:
         x = '교육/문화용품'
    elif x in ['BABY/INFANT', 'BOYS 1', 'BOYS 2', 'Fast Food', 'GIRLS',
       'SPORTS/OUTDOOR', 'VIDEOGAME', '가공식품', '균일가', '기타', '동물병원',
       '드럭스토어', '디저트', '레스토랑', '맛집행사', '먹거리행사', '베이커리', '보석', '분식', '브랜드',
       '브랜드조리', '브랜드푸드몰', '생활잡화', '섬유잡화', '수입브랜드', '시즌토탈', '시즌행사', '아동',
       '아동브랜드', '아동행사', '액세서리', '양식', '웨딩', '일식', '주유소', '중식', '카페',
       '커리어', '컨세션', '트래디셔널', '판매시설', '패스트푸드', '편의시설', '푸드코트',
       '피자', '한식']:
         x = '기타'
    elif x in ['LOCAL김치채소', 'LOCAL잎채소', '가공건식품', '가공란', '가공분말류', '가공양곡', '가공우유',
       '가루', '간식', '간식용과자', '간식조리', '간편간식', '간편요리', '간편조리', '간편조리생선',
       '간편조리식품', '간편조리행사', '간편채소', '감', '감귤', '감미료', '건강기능식품', '건강선물세트',
       '건강식품', '건과', '건과일', '건면', '건어가공', '건어류', '건오징어', '견과류', '계란',
       '계란류', '고등어/삼치', '과실과일', '과자', '과채과일', '과채류', '과채음료', '구색과일', '구이',
       '구이찜조리', '국내산계육', '국내산돈육', '국내산우육', '국산게/갈치', '국산과일', '국산과일선물세트',
       '굴비', '굴비/반건생선선물세트', '규격RTC', '규격생선선물세트', '근채류', '기능성건강', '기능성음료',
       '기타구색생선', '기타국소고기', '기타수산', '기타음주류', '기타조리식품', '기타축산', '기타회',
       '김/초밥', '김가공품', '김선물세트', '김치류', '김치채소', '김치채소류', '껌', '꽁치/가자미',
       '꿀', '나물', '나물류', '냉동HMR', '냉동MS', '냉동가공', '냉동간식', '냉동과일',
       '냉동규격생선', '냉동규격해물', '냉동대용식', '냉동만두', '냉동수산물', '냉장간식MS', '냉장간편식HMR',
       '냉장규격생선', '냉장기타MS', '냉장농산', '냉장면류', '냉장반찬', '냉장수산', '냉장식사HMR',
       '냉장식사MS', '냉장음료', '냉장축산', '냉장햄세트', '농산물', '단무지류', '닭고기', '담배',
       '대구', '대용식', '대중선어', '데일리처분', '도미/민어', '도시락', '돼지고기', '두부', '두유',
       '두채류', '디저트', '딸기', '떡', '라면', '레몬', '마른김', '마른안주', '마른안주선물세트',
       '만두', '망고', '맥주', '먹거리', '메론', '면류', '멸치', '멸치건명태', '멸치황태미역선물세트',
       '묵', '묵/즉석참기름', '미국산소고기', '미역/다시마', '바나나', '반건생선', '반찬', '반찬류',
       '발효유', '밤', '밥류', '밥죽류', '배', '버섯', '버섯류', '베이커리', '병통조림', '복숭아',
       '복합조미료', '분유', '브랜드돼지고기', '브랜드빵', '브랜드한우', '블루베리', '비스켓', '비타민',
       '빵', '뿌리채소', '사과', '새우', '샐러드/간편채소', '샐러드채소', '생선선물세트', '생선초밥',
       '생선회', '생수', '생활한방', '석류', '선물세트', '선식', '선식류', '선어가공', '선어류',
       '소금/설탕류', '소스류', '소시지', '소주', '수박', '수산품', '수입갈치', '수입감귤과일', '수입게',
       '수입고등어', '수입과일', '수입과자', '수입기타과일', '수입기타생선', '수입돈육', '수입돼지고기',
       '수입연어', '수입우육', '수입육', '수입육선물세트', '수입음료', '수입조미', '수입포도', '순대',
       '스낵', '시리얼', '시즌반찬', '시즌차', '식사류', '식용유', '식용유지', '식자재', '씨리얼',
       '아이스크림', '안주', '액상조미료', '약초', '양곡', '양념류', '양념육', '양념채소', '양주',
       '양채류', '어묵맛살', '연체', '열대과일', '열대과일기타', '열대과일선물세트', '열매채소', '엽채류',
       '오렌지', '오렌지과일기타', '오렌지과일선물세트', '오리고기', '오징어', '온라인규격/간편조리생선',
       '온라인돼지고기', '온라인버섯/열매채소', '온라인양념/뿌리채소', '온라인잎/샐러드채소', '온라인잡곡선식',
       '온라인저장과일', '온라인해물연체', '와인', '요구르트', '우유', '우육', '원두커피', '유가공',
       '유제품', '육가공품', '육류', '육포', '음료', '이벤트조리', '이온음료', '이유식/유아간식',
       '인스턴트선물세트', '인스턴트커피', '일반한우', '잎채소', '자두', '자몽', '잡곡', '잡곡류', '장류',
       '잼', '전통간식', '전통과자', '전통주', '젓갈/반찬', '젓갈류', '조개', '조개살', '조기/명태',
       '조리반찬', '조미김', '조미료', '조미선물세트', '조미오징어', '족발', '종합과자', '주류', '쥐치포',
       '즉석구이김', '즉석구이안주', '즉석도정미', '즉석식품', '즉석참기름', '차', '차/커피', '차류',
       '차별화돼지고기', '차선물세트', '차음료', '참외', '천연조미료', '체리', '초콜렛', '초콜릿',
       '축산가공', '치즈', '치즈버터', '친환경과일', '친환경채소', '캔디', '캔디껌류', '커피', '커피류',
       '커피선물세트', '커피음료', '커피차음료', '케첩마요네즈', '콩나물', '쿠키케이크', '키위', '키위/석류',
       '탄산음료', '토마토', '통조림', '튀김', '튀김조리', '특산물', '특산물행사', '파이', '파인애플',
       '포도', '포도/체리', '포장반찬', '포장반찬HMR', '포장젓갈', '프리미엄과자', '피자', '한과',
       '한방식재류', '한방차', '한우선물세트', '해물류', '해물선물세트', '해물연체류', '해조류', '해초',
       '해초류', '햄', '햄소시지', '향신료', '호주산소고기', '홍인삼', '황태', '흰우유']:
         x = '식품'
    elif x in ['개인위생용품', '생리대', '생리용품', '시니어건강용품', '여성용품', '여성위생용품', '위생용품',
       '일반의약외품']:
         x = '의약품'
    elif x in ['DIY욕실용품', 'H&B선물세트', '가정잡화', '건전지', '계절팩', '고양이용품', '공구',
       '교환/보수용품', '구강용품', '구강케어', '국내화장품', '균일가', '그릇/수저', '기능헤어케어',
       '기저귀', '기초화장품', '남성', '남성용품', '남성케어', '네일메이크업', '담배', '덴탈케어',
       '두발용품', '레저취미', '립메이크업', '매트', '면도용품', '물티슈', '미용소품', '미용잡화',
       '미용티슈', '밀대/걸레', '바디용품', '바디케어', '방향/제습/탈취제', '베이비케어', '베이스메이크업',
       '베이직케어', '보수용품', '살충제', '색조화장품', '생리용품', '생활잡화', '선케어', '섬유유연제',
       '세차용품', '세탁세제', '세탁용품', '수납용품', '수예소품', '수예시즌', '수입브랜드', '수족관용품',
       '스킨케어', '시니어건강용품', '시니어위생용품', '식기', '아로마', '아이메이크업', '안전용품',
       '애견용품', '애완/동물용품', '야외/시즌용품', '야외용품', '여성', '여행용품', '오토바이용품',
       '온라인화장품', '욕실용품', '위생용품', '유아동용품', '유아세제', '유아식기', '유아용품',
       '유아위생용품', '음식저장/보관', '음식조리', '일반욕실용품', '일반헤어케어', '일용잡화',
       '자동차 액세서리', '자동차 정비용품', '자동차 청소용품', '자동차 편의용품', '자동차방향제/소취제',
       '자동차장식용품', '자동차처분', '장착용품', '전기용품', '전기전자', '제기용품', '제습제', '조리도구',
       '조리용기', '종량제봉투', '주거세제', '주방세제', '주방시즌/균일가', '주방용품', '주방정리',
       '주방주거세제', '집중케어', '청소욕실용품', '청소용품', '청소편의도구', '카시트/매트', '클렌징',
       '키친타올', '탈취제', '패브릭욕실용품', '표백제', '풋/핸드케어', '프리미엄매트', '핸드/풋케어',
       '헤어스타일링', '헤어케어', '화장지', '화장지/티슈', '화장품', '화장품브랜드샵', '훼이셜케어',
       '휴대폰용품', '휴지통']:
         x = '일상용품'
    elif x in ['TRAVEL', '골프', '골프용품', '구기용품', '낚시', '남성스포츠', '남성스포츠NPB',
       '남성스포츠PB', '남성스포츠특정', '등산', '라켓스포츠', '레저취미', '롤러보드', '수영복', '스키',
       '스포츠', '스포츠ACC', '스포츠NB', '스포츠브랜드편집', '스포츠슈즈', '스포츠웨어', '스포츠의류',
       '아웃도어퍼니처', '야외/시즌용품', '여행용품', '워터스포츠', '윈터스포츠', '자동차장식용품', '자전거',
       '주니어스포츠', '캠핑', '피트니스', '헬스용품']:
         x = '전문스포츠/레저'
    elif x in ['NB구두', '가방', '가방브랜드', '구두', '기타', '남성구두', '남성양말', '남성캐주얼', '덧신',
       '드레스화브랜드', '란제리/내의', '명품', '모자', '미입점행사', '벨트', '샌들슬리퍼', '섬유잡화',
       '슈즈용품', '슈즈행사', '스카프', '스타킹', '스포츠', '시계', '시즌슈즈', '시즌잡화', '썬글라스',
       '아동', '아동슈즈', '아동양말', '아동화브랜드', '액세서리', '양말', '양말류', '양말선물세트',
       '여성구두', '여성양말', '온라인양말', '우산/양산', '우산/양산류', '운동화', '의류잡화', '잡화멀티샵',
       '장갑', '주니어양말', '준보석', '준보석/시계', '지갑', '지갑/벨트', '캐주얼', '캐주얼슈즈',
       '캐주얼화브랜드', '트래디셔널', '피혁브랜드', '피혁잡화', '피혁토탈', '핸드백','교복', '기타의류', '남성 트랜디', '남성베이직', '남성브랜드', '남성시즌행사', '남성어덜트캐주얼',
       '남성이지캐주얼', '남성정장', '남성캐주얼', '남성행사', '디자이너', '란제리', '란제리/내의',
       '란제리브랜드', '마담/어덜트', '명품', '모피/피혁', '미시캐주얼', '병행수입', '복합샵', '블라우스',
       '비즈니스정장', '섬유잡화', '성인내의', '셔츠', '스트리트편집샵', '시티웨어', '시티캐주얼', '아동',
       '아동내의', '아동복합샵', '언더웨어행사', '여성NPB', '여성PB', '여성베이직', '여성이지캐주얼',
       '여성캐주얼브랜드', '여성특정', '유아동NPB', '유아동PB', '유아동특정', '유아브랜드', '유아토들러',
       '의류잡화', '이지캐주얼브랜드', '주니어웨어', '캐주얼', '커리어', '트래디셔널', '행사', '홈웨어']:
         x = '의류/잡화'
 
    return x 

In [ ]:
def map_df(df):
  df['분류체계'] = df['중분류명'].map(map_item)

In [ ]:
map_df(df2014_1_new)
map_df(df2014_2_new)
map_df(df2014_3_new)
map_df(df2014_4_new)
map_df(df2015_1_new)
map_df(df2015_2_new)
map_df(df2015_3_new)
map_df(df2015_4_new)

map_df(df2014_1_exi)
map_df(df2014_2_exi)
map_df(df2014_3_exi)
map_df(df2014_4_exi)
map_df(df2015_1_exi)
map_df(df2015_2_exi)
map_df(df2015_3_exi)
map_df(df2015_4_exi)

In [ ]:
def make_df(df_new, df_exi, i = '2014_1'):
    df1 = pd.DataFrame(df_new.groupby('분류체계')['구매금액'].sum())
    df1 = df1.rename({'구매금액':'신규고객구매금액'}, axis=1)
    df1['기존고객구매금액'] = pd.DataFrame(df_exi.groupby('분류체계')['구매금액'].sum())['구매금액'].values
    df1['분기'] = i
    df1 = df1.reset_index()
    return df1

def make_count_df(df_new, df_exi, i = '2014_1'):
    df1 = pd.DataFrame(df_new.groupby('분류체계')['구매금액'].count())
    df1 = df1.rename({'구매금액':'신규고객구매횟수'}, axis=1)
    df1['기존고객구매횟수'] = pd.DataFrame(df_exi.groupby('분류체계')['구매금액'].count())['구매금액'].values
    df1['분기'] = i
    df1 = df1.reset_index()
    return df1

def append_make_count_df(df_append, df_new, df_exi, i = '2014_1'):
    a = make_count_df(df_new, df_exi, i =i)
    df_append = df_append.append(a)
    return df_append

def append_make_df(df_append, df_new, df_exi, i = '2014_1'):
    a = make_df(df_new, df_exi, i =i)
    df_append = df_append.append(a)
    return df_append

In [ ]:
data = pd.DataFrame()
data = append_make_df(data, df2014_1_new, df2014_1_exi, i = '2014_1')
data = append_make_df(data, df2014_2_new, df2014_2_exi, i = '2014_2')
data = append_make_df(data, df2014_3_new, df2014_3_exi, i = '2014_3')
data = append_make_df(data, df2014_4_new, df2014_4_exi, i = '2014_4')

data_count = pd.DataFrame()
data_count = append_make_count_df(data_count, df2014_1_new, df2014_1_exi, i = '2014_1')
data_count = append_make_count_df(data_count, df2014_2_new, df2014_2_exi, i = '2014_2')
data_count = append_make_count_df(data_count, df2014_3_new, df2014_3_exi, i = '2014_3')
data_count = append_make_count_df(data_count, df2014_4_new, df2014_4_exi, i = '2014_4')

In [ ]:
plt.figure(figsize=(20,30))
sns.barplot(x = '분기', y = '기존고객구매금액', data = data, hue='분류체계', color='green')
sns.barplot(x = '분기', y = '신규고객구매금액', data = data, hue='분류체계', color='yellow')
plt.show()

In [ ]:
data1 = pd.DataFrame()
data1 = append_make_df(data1, df2015_1_new, df2015_1_exi, i = '2015_1')
data1 = append_make_df(data1, df2015_2_new, df2015_2_exi, i = '2015_2')
data1 = append_make_df(data1, df2015_3_new, df2015_3_exi, i = '2015_3')
data1 = append_make_df(data1, df2015_4_new, df2015_4_exi, i = '2015_4')

data_count1 = pd.DataFrame()
data_count1 = append_make_count_df(data_count1, df2015_1_new, df2015_1_exi, i = '2015_1')
data_count1 = append_make_count_df(data_count1, df2015_2_new, df2015_2_exi, i = '2015_2')
data_count1 = append_make_count_df(data_count1, df2015_3_new, df2015_3_exi, i = '2015_3')
data_count1 = append_make_count_df(data_count1, df2015_4_new, df2015_4_exi, i = '2015_4')

In [ ]:
plt.figure(figsize=(20,30))
sns.barplot(x = '분기', y = '기존고객구매금액', data = data1, hue='분류체계', color='blue')
sns.barplot(x = '분기', y = '신규고객구매금액', data = data1, hue='분류체계', color='yellow')
plt.show()

In [ ]:
def plt_count_df(da, color='green'):
    plt.figure(figsize=(20,30))
    plt.rc('font', family='NanumGothic')
    sns.barplot(x = '분기', y = '기존고객구매횟수', data = da, hue='분류체계', color=color)
    sns.barplot(x = '분기', y = '신규고객구매횟수', data = da, hue='분류체계', color='yellow')
    plt.show()

In [ ]:
plt_count_df(data_count)

In [ ]:
plt_count_df(data_count1)

In [ ]:
data_count_drop = data_count[data_count['분류체계'] != '기타']
data_count1_drop = data_count1[data_count1['분류체계'] != '기타']
data_count_drop2 = data_count_drop[data_count_drop['분류체계'] != '식품']
data_count1_drop2 = data_count1_drop[data_count1_drop['분류체계'] != '식품']

In [ ]:
plt_count_df(data_count_drop2)